In [1]:
sc.install_pypi_package('pandas==1.3.5')
sc.install_pypi_package('boto3')
sc.install_pypi_package('pyspark==2.3.4')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1687300297410_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…






    ERROR: Command errored out with exit status 1:
     command: /mnt/yarn/usercache/livy/appcache/application_1687300297410_0001/container_1687300297410_0001_01_000001/tmp/1687302430604-0/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/mnt/tmp/pip-install-l3jhof9e/pyspark/setup.py'"'"'; __file__='"'"'/mnt/tmp/pip-install-l3jhof9e/pyspark/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /mnt/tmp/pip-pip-egg-info-8ddadlrp
         cwd: /mnt/tmp/pip-install-l3jhof9e/pyspark/
    Complete output (24 lines):
    Could not import pypandoc - required to package PySpark
    Couldn't find index page for 'pypandoc' (maybe misspelled?)
    No local packages or working download links found for pypandoc
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/mnt/tmp/pip-install-l3jhof9e/pys

In [2]:
import pandas
import re
import tempfile
import os
import gzip
import shutil
import subprocess
from pyspark.sql.functions import udf, col, lit
from pyspark.sql import functions as F

"""
Import Parquet As a DataFrame
"""

##Read in parquet file from public S3 bucket
parquet_s3 = "s3://steichenetalpublicdata/analyzed_sequences/parquet"
df_spark = spark.read.parquet(parquet_s3)

##Verify count 
df_spark.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1112960406

In [3]:
class Query():
    
    '''An example query class to hold query parameters'''
    
    def __init__(self,q_name,length='',v_fam="",v_gene="",d_gene="",j_gene="",regex="",ez_donor=""):
        self.query_name = q_name
        self.v_fam = v_fam
        self.v_gene = v_gene
        self.j_gene = j_gene
        self.d_gene = d_gene
        self.ez_donor = ez_donor
        
        if not length:
            raise Exception("Length must be supplied")
        self.length = length
        self.regular_expression = regex
    
    
    
    def apply(self,df):
        
        '''Apply function will take in spark dataframe and apply query parameters to it if they exist
        
           Returns a filtered dataframe
        '''
        self.queried_dataframe = ""
        
        ##Lets get length
        
        self.queried_dataframe = df.filter(F.length(df.cdr3_aa) > self.length)
        
        ##If the rest of these were specified, add them to the filter
        if self.v_fam:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.v_fam == self.v_fam)
        
        if self.v_gene:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.v_gene == self.v_gene)
     
        if self.d_gene:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.d_gene == self.d_gene)       
        
        if self.j_gene:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.j_gene == self.j_gene)       
        

        if self.regular_expression:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.cdr3_aa.rlike(self.regular_expression))
        
        if self.ez_donor:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.ez_donor == self.ez_donor)
            
        print("Found {} sequences".format(self.queried_dataframe.count()))
        return self.queried_dataframe
        
    
def write_out_hadoop_to_local(df, s3):
    '''Because spark is distributed, writing out results is done by each worker node. As a result, it breaks up the results into partions. 
    
    This function will search the hadoop file system and concatenate the parted file back to a single CSV file
    
    params df - the quried dataframe
    s3 - the local file path (make sure its chmod 777)
    '''
    
    #Output Name
    output_s3 = s3
    
    #A temporary file to write out Hadoop CSV
    t = tempfile.NamedTemporaryFile(delete=False)
    

    #Write out the CSV
    df.write.csv(t.name,mode='overwrite')
    with open(output_s3,'w') as f:
        f.write(",".join(df.columns)+'\n')
        
        print("Running HDFS concatenate\n{} > {}".format(' '.join(['hdfs','dfs','-cat','{}/*csv'.format(t.name)]),output_s3))
        subprocess.Popen(['hdfs','dfs','-cat','{}/*csv'.format(t.name)],stdout=f)
        

    
    print('Wrote CSV to {}'.format(output_s3))

my_query_1 = Query('BG18_search.1',length=21)
my_query_2 = Query('BG18_search.2',d_gene='IGHD3-3',length=21)
my_query_3 = Query('BG18_search.3',d_gene='IGHD3-3',length=21, regex=r'FGV')
my_query_4_7 = Query('BG18_search.4.7',d_gene='IGHD3-3',length=21, regex=r'^......FGV')
my_query_4_8 = Query('BG18_search.4.8',d_gene='IGHD3-3',length=21, regex=r'^.......FGV')
my_query_4_9 = Query('BG18_search.4.9',d_gene='IGHD3-3',length=21, regex=r'^........FGV')
my_query_5_7 = Query('BG18_search.5.7',d_gene='IGHD3-3',length=21, regex=r'^......FGV....E')
my_query_5_8 = Query('BG18_search.5.8',d_gene='IGHD3-3',length=21, regex=r'^.......FGV....E')
my_query_5_9 = Query('BG18_search.5.9',d_gene='IGHD3-3',length=21, regex=r'^........FGV....E')
my_query_6 = Query('BG18_search.6',d_gene='IGHD3-3',length=21, regex=r'FGV....E')
my_query_7 = Query('BG18_search.7',d_gene='IGHD3-3',length=1)

#To run query, pass the input object from above to apply

queried1_df = my_query_1.apply(df_spark)
queried2_df = my_query_2.apply(df_spark)
queried3_df = my_query_3.apply(df_spark)
queried4_7_df = my_query_4_7.apply(df_spark)
queried4_8_df = my_query_4_8.apply(df_spark)
queried4_9_df = my_query_4_9.apply(df_spark)
queried5_7_df = my_query_5_7.apply(df_spark)
queried5_8_df = my_query_5_8.apply(df_spark)
queried5_9_df = my_query_5_9.apply(df_spark)
queried6_df = my_query_6.apply(df_spark)
queried7_df = my_query_7.apply(df_spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found 57075 sequences

In [32]:
pandas1_df = queried1_df.select('_id','ez_donor').toPandas()
pandas2_df = queried2_df.select('_id','ez_donor').toPandas()
pandas3_df = queried3_df.select('_id','ez_donor').toPandas()
pandas47_df = queried4_7_df.select('_id','ez_donor').toPandas()
pandas48_df = queried4_8_df.select('_id','ez_donor').toPandas()
pandas49_df = queried4_9_df.select('_id','ez_donor').toPandas()
pandas57_df = queried5_7_df.select('_id','ez_donor').toPandas()
pandas58_df = queried5_8_df.select('_id','ez_donor').toPandas()
pandas59_df = queried5_9_df.select('_id','ez_donor').toPandas()
pandas6_df = queried6_df.select('_id','ez_donor').toPandas()
pandas7_df = queried7_df.select('_id','ez_donor').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
pandas4_df = pandas.concat([pandas47_df,pandas48_df,pandas49_df])
pandas5_df = pandas.concat([pandas57_df,pandas58_df,pandas59_df])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
counts = pandas1_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
counts = pandas2_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

In [ ]:
counts = pandas3_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

In [ ]:
counts = pandas4_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

In [ ]:
counts = pandas5_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

In [5]:
counts = pandas6_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   ez_donor     normal
9         1  22.440817
8        10  52.467923
1        11   4.229708
6        12  25.441762
10       13  26.772067
7        14  14.038361
11        2  75.570549
13        3  65.397028
12        4  57.016984
3         5  18.864662
2         6  13.998922
0         7  11.643994
4         8  30.660714
5         9  30.799921

In [ ]:
counts = pandas7_df.groupby('ez_donor').count().rename({'_id':'count'},axis=1).sort_values('count')
df_count = df_spark.groupby('ez_donor').count().toPandas()
merged = counts.reset_index().merge(df_count, on='ez_donor')
merged['normal'] = merged['count_x']/merged['count_y']*1000000
merged.sort_values('ez_donor')[['ez_donor','normal']]